## Exercise 6. XGBoost

1. Use the code below to download and load the Higgs dataset. It's a binary classification dataset.
2. Train a logistic regression classifier and compute its accuracy on the test set.
3. Can you beat that accuracy using an XGBoost classifier?

[XGBoost Python docs](https://xgboost.readthedocs.io/en/latest/python/python_intro.html)

In [1]:
import numpy as np
import cudf
import cuml
from cuml.linear_model import LogisticRegression
from cuml import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

In [2]:
# download the dataset
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

!ls 
gdf = cudf.read_csv('/users/class192/scratch/SummerSchool2020/topics/ml_rapids/HIGGS.csv.gz', dtypes=[np.float32] * 29, header=None )
gdf.head()

cpu_roundtrips.PNG	     ex5.ipynb		   rapids_comp_pandas.png
cuml_algo.png		     ex6.ipynb		   rapids_comp_scikit.png
dask.PNG		     ex7.ipynb		   rapids_improvement.PNG
datasci_benchmark.PNG	     ex8.ipynb		   rapids_overview.PNG
datasci_day_in_the_life.PNG  ferrari.jpg	   rapids_perf.png
datascience.png		     kaggle_chollet.PNG    rapids.png
datascience_stages.png	     mlcourse.py	   rapids_p.png
datasci_over_the_years.PNG   nvidia_dl.png	   slides.html
dl_frameworks.png	     octavia.jpg	   solutions_1_2.ipynb
ex1.ipynb		     profile_opt.png	   tpc1.PNG
ex2.ipynb		     pystack.png	   tpc2.PNG
ex3.ipynb		     rapids_comp_cudf.png  umap_img
ex4.ipynb		     rapids_comp_cuml.png


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487


In [3]:
X, y = gdf.iloc[:,1:], gdf.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [11]:
mu = X_train.mean(keep_dims=True)
sigma = X_train.std(keep_dims=True)


clf = LogisticRegression()
clf.fit(X_train, y_train)
print(f'train: {clf.score(X_train, y_train)}, test: {clf.score(X_test, y_test)}')

train: 0.5351678133010864, test: 0.5357400178909302


In [6]:
y_pred = clf.predict(X_test)

In [4]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

In [ ]:
params = {}
model = xgb.train(params, dtrain)
pred = model.predict(dtest)
cuml.metrics.accuracy_score(y_test, prediction > 0.5)

[11:03:35] WARNING: /conda/conda-bld/xgboost_1591204502249/work/src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
